In [1]:
#%%
import sys
# sys.path.append("/home/baokhanh/")
import pandas as pd
import numpy as np
import os
import calendar
import psycopg2
from pathlib import Path
from loguru import logger
from dotenv import load_dotenv
from datetime import datetime, timedelta
from unidecode import unidecode
from avay_bq import AvayBQServiceAcc

from slack import WebClient
from slack.errors import SlackApiError

import calendar
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
from math import floor, ceil

from da_utils.client.google_sheets_client import GoogleSheetsClient
from da_utils.repository.google_sheets.google_sheets_repository import GoogleSheetsService

from da_utils.client.drive_client import GoogleDriveClient
from da_utils.repository.drive.drive_repository import GoogleDriveService

from da_utils.client.slack_client import SlackClient
from da_utils.repository.slack.slack_repository import SlackRepository

from da_utils.client.bigquery_client import BigqueryClient
from da_utils.repository.bigquery.bigquery_repository import BigqueryService

load_dotenv()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 2000

In [20]:

drive_client = GoogleDriveClient()
drive_repo = GoogleDriveService(drive_client)

ggsheet_client = GoogleSheetsClient()
ggsheet_repo = GoogleSheetsService(ggsheet_client)

slack_client = SlackClient(token=os.environ.get("SLACK_BOT_AUTHEN"))
slack_repo = SlackRepository(client=slack_client)

avay_bq_acc = AvayBQServiceAcc()

sa_ana = Path('/home/linhnguyen/.config/ts-dp-v2.json')
adc = Path('/home/linhnguyen/.config/gcloud/application_default_credentials.json')
bq_client = BigqueryClient(billing_project_id='prj-ts-p-analytic-8057',  adc_file_path= adc) #sa_file_path = sa_ana) #
bq_repo = BigqueryService(bq_client)

bot_auth_token = os.environ.get("SLACK_BOT_AUTHEN")
user_auth_token = os.environ.get("SLACK_USER_AUTHEN")
clientSlack = WebClient(token = bot_auth_token)

linh_id = "U047QMYB9TQ"
channel_id = "C04MYU3L7LN"
bot_id = "U0480LX5468"

In [6]:
def dayrange_recon(day_from_curr: int = 45) -> tuple:
    curr_time = datetime.now()
    recon_time =(curr_time - timedelta(day_from_curr))
    _, end_date = calendar.monthrange(int(recon_time.strftime("%Y")), int(recon_time.strftime("%m")))

    recon_month = recon_time.strftime("%Y-%m")
    start_date = recon_time.strftime("%Y-%m") + "-" + "01"
    end_date = recon_time.strftime("%Y-%m") + "-" + str(end_date)
    return recon_month, start_date, end_date

In [7]:
recon_month, start_date, end_date = dayrange_recon()
recon_month

'2023-08'

In [8]:
# create directory
path = '/home/linhnguyen/04.Reconcile/06.TPB/nonsms_issued_card/'

import os

if not os.path.exists(f'{path}{recon_month}'):
    os.makedirs(f'{path}{recon_month}')

# GET CARD DATA

In [25]:
query = f""" 
WITH smv as (
  select 
  smv.*
  , DENSE_RANK() OVER (PARTITION BY lead_unique_token ORDER BY smv_issued_at ASC) AS rank
  from `prj-ts-p-analytic-8057.shared_tpb.tpb_smv_dop_map` smv
)

, tbl1 AS (
  SELECT distinct 
  application_id,
  CAST(JSON_VALUE(result_data, '$.id') AS INTEGER) AS offer_qualifications_id
  FROM `prj-ts-p-lakehouse-8435.dop.application_services`
)
, tbl2 AS (
  SELECT DISTINCT 
  id as offer_qualifications_id
  , CAST(JSON_VALUE(result, '$.other.lead_id') AS INTEGER) AS telco_lead_id
  FROM `prj-ts-p-lakehouse-8435.v1_proxy.offer_qualifications` 
)

, mapping AS (
  SELECT 
  tbl1.application_id
  , tbl2.telco_lead_id
  FROM tbl1 JOIN tbl2 USING(offer_qualifications_id)
)
    SELECT DISTINCT 
        dop.lead_unique_token
        , dop.application_id
        , app.lead_phone_number
        , app.lead_product_code
        , app.lead_telco_code
        , mapping.telco_lead_id 
        , cif
        , card_no
        , app.lead_created_at
        , smv_issued_at
        , smv_active_at
        , FORMAT_DATE('%Y-%m', DATE(smv.smv_issued_at)) as issue_month
        , FORMAT_DATE('%Y-%m', DATE(smv.smv_active_at)) as active_month
        , dop.mkt_channel
        , dop.utm_campaign
        , dop.mkt_channel_group AS channel
        , smv.smv_card_category as card_name
        , CASE  WHEN smv.smv_card_category LIKE '%SILVER%' THEN 'SILVER'
                WHEN smv.smv_card_category LIKE '%GOLD%' THEN 'GOLD'
                END AS card_class
        , dop.has_card_issued
        , dop.has_card_active
    FROM `prj-ts-p-analytic-8057.shared_tpb.tpb_dop_mapping` dop
    JOIN (select * from smv where rank=1) smv
        ON dop.lead_unique_token = smv.lead_unique_token
    JOIN `prj-ts-p-lakehouse-8435.dop.applications` app 
        ON dop.application_id = app.id
    LEFT JOIN mapping USING (application_id)
    WHERE 1=1
        AND has_card_issued = 1
        AND (dop.utm_campaign != 'referral01' OR dop.utm_campaign is null)
"""

df = bq_repo.get_data_from_query_into_pandas(query_str=query)

In [26]:
df.head()

,lead_unique_token,application_id,lead_phone_number,lead_product_code,lead_telco_code,telco_lead_id,cif,card_no,lead_created_at,smv_issued_at,smv_active_at,issue_month,active_month,mkt_channel,utm_campaign,channel,card_name,card_class,has_card_issued,has_card_active
0,UiK9Z-aD,5668456,AdfNSLLA6T98lzApZ+Ec7wDAXgy6q4AaK8peDP8=,tpb_01,vinaphone,<NA>,10298418,498796xxxxxx7457,2023-04-26 11:12:23+00:00,2023-04-26 12:22:55,2023-05-15,2023-04,2023-05,COCCOC_ADS,search,PERFORMANCE MARKETING,VISA EVO SILVER CARD (9023),SILVER,1,1
1,kRXu3FGC,6142115,AVc0p1U08OXDV0DeQrkjOZYDKc2InSaSd9baYYA=,tpb_01,viettel,<NA>,04411392,498796xxxxxx3146,2023-07-09 20:48:23+00:00,2023-07-09 21:56:56,2023-07-18,2023-07,2023-07,AFFAT,None,PERFORMANCE MARKETING,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,1
2,QiK91Wtx,6117047,AQYzD4uIMvkMAl/ggZJQLxCWoG+lpM86mz5zqm0=,tpb_01,viettel,<NA>,03516071,498796xxxxxx5436,2023-07-06 09:05:54+00:00,2023-07-06 10:28:07,2023-07-06,2023-07,2023-07,AFFAT,None,PERFORMANCE MARKETING,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,1
3,YRFesCXN,5637305,AaLbcKWB9SS5xSOwCLFO787u8N2/Z+X07gj8MvE=,tpb_01,viettel,<NA>,05813973,498796xxxxxx7637,2023-04-21 12:25:21+00:00,2023-04-21 12:42:58,2023-04-21,2023-04,2023-04,CLICKBUY,offline_clb_mass_070323,EVO PARTNER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,1
4,aRFesiU5,5653671,AexcB8TR04nvSKDLxy0JTds0enjwb5pkIdtLJIg=,tpb_01,viettel,<NA>,01726884,498796xxxxxx4730,2023-04-24 08:31:21+00:00,2023-04-24 09:38:28,2023-04-27,2023-04,2023-04,AFFHY,None,PERFORMANCE MARKETING,VISA EVO SILVER CARD (9023),SILVER,1,1


In [27]:
def leadgen_pricing(df, recon_month):
    df['lead_source'] = np.select([df['mkt_channel'].str.contains('WITH OFFER'), 
                                         df['mkt_channel'].str.contains('WITHOUT OFFER'),
                                         ~df['mkt_channel'].isna()],
                                        ['SMS WITH OFFER', 
                                         'SMS WITHOUT OFFER', 
                                         'EVO/DIGITAL'])

    df['lead_cost'] = np.select([df['card_class'] == 'SILVER', 
                                       df['card_class'] == 'GOLD'], 
                                      [900000,
                                       900000],
                                      default=np.nan)

    df['lead_created_at'] = pd.to_datetime(df['lead_created_at'], format='%Y-%m-%d').dt.tz_localize(None)

    df = df.query("(card_class in ['GOLD', 'SILVER']) & (issue_month == @recon_month)")  

    return df

tpb_card = leadgen_pricing(df, recon_month=recon_month)

In [28]:
tpb_card[['channel','lead_source']].drop_duplicates()

,channel,lead_source
66,EVO PARTNER,EVO/DIGITAL
78,PERFORMANCE MARKETING,EVO/DIGITAL
108,SMS WITHOUT OFFER,SMS WITHOUT OFFER
154,OTHER,EVO/DIGITAL
254,EVO BRANDING,EVO/DIGITAL
101457,SMS WITH OFFER,SMS WITH OFFER


In [29]:
tpb_card[['card_class','lead_cost']].drop_duplicates()

,card_class,lead_cost
66,SILVER,900000.0
78,GOLD,900000.0


In [30]:
tpb_card[['issue_month']].drop_duplicates()

,issue_month
66,2023-08


In [31]:
df1 = tpb_card.groupby(['issue_month', 'lead_source', 'card_class']).agg(
    issue_cnt=('has_card_issued', 'sum'),
    active_cnt=('has_card_active', 'sum'),
    lead_cost=('lead_cost', 'sum')
).reset_index()

df1 = df1[df1['lead_source'].notna()].sort_values(['issue_month', 'lead_source', 'card_class'], ascending=False)

df1 

,issue_month,lead_source,card_class,issue_cnt,active_cnt,lead_cost
5,2023-08,SMS WITHOUT OFFER,SILVER,125,107,1.125000e+08
4,2023-08,SMS WITHOUT OFFER,GOLD,128,116,1.152000e+08
3,2023-08,SMS WITH OFFER,SILVER,10,9,9.000000e+06
2,2023-08,SMS WITH OFFER,GOLD,15,13,1.350000e+07
1,2023-08,EVO/DIGITAL,SILVER,2927,2599,2.634300e+09
0,2023-08,EVO/DIGITAL,GOLD,2181,1978,1.962900e+09


# UPDATE GDRIVE

In [32]:
#xlsx
tpb_card.to_excel(f"{path}{recon_month}/tpb_leadgen_recon_{recon_month}.xlsx", index=False)

In [33]:
def create_or_get_folder(recon_month, parent_folder_id):
    # Check if folder exists
    folder_check = drive_repo.list_folder_names(parent_folder_id =  parent_folder_id)
    if recon_month not in folder_check:
        folder_id = drive_repo.create_folder(folder_name= recon_month, parent_folder_id =  parent_folder_id)
    else:
        folder_id = drive_repo.get_folders(folder_name= recon_month, parent_folder_id =  parent_folder_id)
    return folder_id[0]

def create_or_get_workbook(parent_folder_id, wb_name):
    wb_check = drive_repo.list_file_names(parent_folder_id= parent_folder_id)
    if wb_name not in wb_check:
        wb = ggsheet_repo.create_spreadsheet(spreadsheet_name = wb_name,parent_folder_id = parent_folder_id)
    else:
        spreadsheet_key = drive_repo.get_files(file_name = wb_name,parent_folder_id = parent_folder_id)
        wb = ggsheet_repo.open_spreadsheet(spreadsheet_key = spreadsheet_key[0])
    return wb

def create_or_update_sheet(wb, new_sheet, input_df):
    sheet_names = [worksheet.title for worksheet in wb.worksheets()]
    if new_sheet not in sheet_names:
        wb.add_worksheet(title=new_sheet, rows=100, cols=20)
        ws = wb.worksheet(new_sheet)
        ggsheet_repo.write_df_to_sheet(worksheet = ws, df_to_write = input_df, starting_cell='A1')
    else:
        ws = wb.worksheet(new_sheet)
        ws.clear()
        ggsheet_repo.write_df_to_sheet(worksheet = ws, df_to_write = input_df, starting_cell='A1')
    if 'Sheet1' in sheet_names: 
        wb.del_worksheet(wb.worksheet('Sheet1')) 
    else:
        pass
    return

In [34]:
parent_folder_id = '11QuXavTTWfBQXAB8m-n9JFBTavdY5uEU'
folder_id = create_or_get_folder(recon_month=recon_month, parent_folder_id=parent_folder_id)
TPB_MBF_wb = create_or_get_workbook(parent_folder_id = folder_id, wb_name = "TPB_leadgen_issued_" + recon_month)
create_or_update_sheet(wb = TPB_MBF_wb, new_sheet = "Details", input_df = tpb_card)

In [35]:
summary_data = tpb_card.groupby(['issue_month', 'lead_source', 'card_class']).agg({'has_card_issued':'sum', 'lead_cost':'sum'}).reset_index()
Summary_by_channel = tpb_card.groupby(['issue_month', 'lead_source', 'channel', 'card_class']).agg({'has_card_issued':'sum', 'lead_cost':'sum'}).reset_index()

create_or_update_sheet(TPB_MBF_wb, "Summary", summary_data)
create_or_update_sheet(TPB_MBF_wb, "Summary_by_channel (internal)", Summary_by_channel)

# UPDATE DISBURSAL

In [36]:
tpb_sms = tpb_card[tpb_card['lead_source'] == 'SMS WITH OFFER']
tpb_sms.head()


,lead_unique_token,application_id,lead_phone_number,lead_product_code,lead_telco_code,telco_lead_id,cif,card_no,lead_created_at,smv_issued_at,smv_active_at,issue_month,active_month,mkt_channel,utm_campaign,channel,card_name,card_class,has_card_issued,has_card_active,lead_source,lead_cost
101457,eRFe97N7,6223471,AYNr4ffXPcUrMHaAa1GNpE/8+0XMpXbasMWYfZE=,tpb_01,mobifone,3516920,11976620,498796xxxxxx7087,2023-07-21 09:47:36,2023-08-18 10:06:02,2023-08-18,2023-08,2023-08,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,1,SMS WITH OFFER,900000.0
101788,aRFe_O5a,6266315,AVZZvxUuxmayAxuTwzSMCMSRHgga92+JfG0HUvQ=,tpb_01,mobifone,3519374,11811531,498796xxxxxx9574,2023-07-26 20:36:53,2023-08-01 16:44:09,2023-08-04,2023-08,2023-08,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,1,SMS WITH OFFER,900000.0
102138,YRFfAwNu,6316141,Aes2QNkwisrMSVy0+5qrQ8LZHlb18foqaDWax7s=,tpb_01,mobifone,3520607,11814133,498796xxxxxx6464,2023-08-01 23:35:01,2023-08-02 01:34:43,2023-08-04,2023-08,2023-08,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,1,SMS WITH OFFER,900000.0
102587,cRFe_YM_,6271079,AXUr79XzNUQfoDl4RP2HJrCwxfW4Gn+INzfYw3o=,tpb_01,mobifone,3519602,03030167,498796xxxxxx8285,2023-07-27 13:30:57,2023-08-09 16:00:28,2023-08-16,2023-08,2023-08,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,1,SMS WITH OFFER,900000.0
102670,QiK97rHg,6220574,AT1jNHQtfhCK4RwwlejWzJ5aMgUFv+N5DrutT60=,tpb_01,mobifone,3516677,02009151,498796xxxxxx2147,2023-07-20 21:21:02,2023-08-05 09:00:08,NaT,2023-08,None,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,0,SMS WITH OFFER,900000.0


In [37]:
tpb_sms.count()

lead_unique_token    25
application_id       25
lead_phone_number    25
lead_product_code    25
lead_telco_code      25
telco_lead_id        25
cif                  25
card_no              25
lead_created_at      25
smv_issued_at        25
smv_active_at        22
issue_month          25
active_month         22
mkt_channel          25
utm_campaign          0
channel              25
card_name            25
card_class           25
has_card_issued      25
has_card_active      25
lead_source          25
lead_cost            25
dtype: int64

## get ts_lead_id

In [38]:
v1_query = f"""
select distinct
CAST(id as INT64) as lead_id
, phone_number
, CAST(telco_lead_id as INT64) as telco_lead_id
, date(created_at) as lead_created_at
from `prj-ts-p-lakehouse-8435.v1_proxy.phone_infos`
where true
and bank_code = 'tpbank'
                        ;
"""
v1_df = bq_repo.get_data_from_query_into_pandas(query_str= v1_query)

In [39]:
def generate_tpb_sms_map(tpb_sms, v1_df):
    def to_categorical(tpb_sms_map):
        return pd.merge(tpb_sms_map, 
                         v1_df[['phone_number', 'telco_lead_id', 'lead_id']],
                         how='left',
                         left_on=['telco_lead_id', 'lead_phone_number'],
                         right_on=['telco_lead_id', 'phone_number']).dropna(subset=['telco_lead_id']).drop_duplicates()

    tpb_sms_map = to_categorical(tpb_sms)

    def to_categorical_na(tpb_sms):
        tpb_sms_na = tpb_sms[tpb_sms['telco_lead_id'].isna()].drop(columns=['telco_lead_id'])
        return pd.merge(tpb_sms_na, 
                         v1_df[['phone_number', 'telco_lead_id', 'lead_id']],
                         how='left',
                         left_on='lead_phone_number',
                         right_on='phone_number').drop_duplicates()

    tpb_sms_map_na = to_categorical_na(tpb_sms)

    def combine_tpb_sms_maps(tpb_sms_map, tpb_sms_map_na):
        tpb_sms_map_full = pd.concat([tpb_sms_map, tpb_sms_map_na])
        return tpb_sms_map_full.drop_duplicates()

    tpb_sms_map_full = combine_tpb_sms_maps(tpb_sms_map, tpb_sms_map_na)

    return tpb_sms_map_full

tpb_sms_map = generate_tpb_sms_map(tpb_sms, v1_df)

display(tpb_sms_map.head())

,lead_unique_token,application_id,lead_phone_number,lead_product_code,lead_telco_code,telco_lead_id,cif,card_no,lead_created_at,smv_issued_at,smv_active_at,issue_month,active_month,mkt_channel,utm_campaign,channel,card_name,card_class,has_card_issued,has_card_active,lead_source,lead_cost,phone_number,lead_id
0,eRFe97N7,6223471,AYNr4ffXPcUrMHaAa1GNpE/8+0XMpXbasMWYfZE=,tpb_01,mobifone,3516920,11976620,498796xxxxxx7087,2023-07-21 09:47:36,2023-08-18 10:06:02,2023-08-18,2023-08,2023-08,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,1,SMS WITH OFFER,900000.0,AYNr4ffXPcUrMHaAa1GNpE/8+0XMpXbasMWYfZE=,14222434
1,aRFe_O5a,6266315,AVZZvxUuxmayAxuTwzSMCMSRHgga92+JfG0HUvQ=,tpb_01,mobifone,3519374,11811531,498796xxxxxx9574,2023-07-26 20:36:53,2023-08-01 16:44:09,2023-08-04,2023-08,2023-08,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,1,SMS WITH OFFER,900000.0,AVZZvxUuxmayAxuTwzSMCMSRHgga92+JfG0HUvQ=,14223442
2,YRFfAwNu,6316141,Aes2QNkwisrMSVy0+5qrQ8LZHlb18foqaDWax7s=,tpb_01,mobifone,3520607,11814133,498796xxxxxx6464,2023-08-01 23:35:01,2023-08-02 01:34:43,2023-08-04,2023-08,2023-08,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,1,SMS WITH OFFER,900000.0,Aes2QNkwisrMSVy0+5qrQ8LZHlb18foqaDWax7s=,14224060
3,cRFe_YM_,6271079,AXUr79XzNUQfoDl4RP2HJrCwxfW4Gn+INzfYw3o=,tpb_01,mobifone,3519602,03030167,498796xxxxxx8285,2023-07-27 13:30:57,2023-08-09 16:00:28,2023-08-16,2023-08,2023-08,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,1,SMS WITH OFFER,900000.0,AXUr79XzNUQfoDl4RP2HJrCwxfW4Gn+INzfYw3o=,14223540
4,QiK97rHg,6220574,AT1jNHQtfhCK4RwwlejWzJ5aMgUFv+N5DrutT60=,tpb_01,mobifone,3516677,02009151,498796xxxxxx2147,2023-07-20 21:21:02,2023-08-05 09:00:08,NaT,2023-08,None,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,0,SMS WITH OFFER,900000.0,AT1jNHQtfhCK4RwwlejWzJ5aMgUFv+N5DrutT60=,14222342


In [40]:
tpb_sms_map.count()

lead_unique_token    25
application_id       25
lead_phone_number    25
lead_product_code    25
lead_telco_code      25
telco_lead_id        25
cif                  25
card_no              25
lead_created_at      25
smv_issued_at        25
smv_active_at        22
issue_month          25
active_month         22
mkt_channel          25
utm_campaign          0
channel              25
card_name            25
card_class           25
has_card_issued      25
has_card_active      25
lead_source          25
lead_cost            25
phone_number         25
lead_id              25
dtype: int64

In [41]:
def preprocess_disbursed(input_df):
    output_df = input_df.copy()
    output_df['bank_code'] = 'tpbank'
    output_df['telco_lead_id'] = output_df['telco_lead_id'].astype(int)
    output_df['lead_id'] = output_df['lead_id'].astype(int, errors='ignore')
    output_df['lead_source'] = output_df['lead_source'].astype(str)
    output_df['phone_number'] = output_df['lead_phone_number'].astype(str)
    output_df['nid'] = pd.NA
    output_df['lead_unique_token'] = output_df['lead_unique_token'].astype(str)
    output_df['lead_created_date'] = pd.to_datetime(output_df['lead_created_at']).dt.date
    output_df['channel'] = 'esms'
    output_df['product_code'] = output_df['lead_product_code'].astype(str)
    output_df['card_name'] = output_df['card_name'].astype(str)
    output_df['card_class'] = output_df['card_class'].astype(str)
    output_df['telco_code'] = output_df['lead_telco_code'].astype(str)
    output_df['issued_date'] = pd.to_datetime(output_df['smv_issued_at']).dt.date
    output_df['activated_date'] = pd.to_datetime(output_df['smv_active_at']).dt.date
    output_df['commission'] = output_df['lead_cost'].astype(float)
    output_df['other'] = pd.NA
    output_df = output_df[['bank_code', 'telco_lead_id', 'lead_id', 'lead_source', 'phone_number',
                           'nid', 'lead_unique_token', 'lead_created_date', 'channel', 'product_code',
                           'card_name', 'card_class', 'telco_code', 'issued_date', 'activated_date', 
                           'commission', 'other']]
    return output_df


In [42]:
disbursal_df = preprocess_disbursed(tpb_sms_map)

In [43]:
disbursal_df.head()

,bank_code,telco_lead_id,lead_id,lead_source,phone_number,nid,lead_unique_token,lead_created_date,channel,product_code,card_name,card_class,telco_code,issued_date,activated_date,commission,other
0,tpbank,3516920,14222434,SMS WITH OFFER,AYNr4ffXPcUrMHaAa1GNpE/8+0XMpXbasMWYfZE=,<NA>,eRFe97N7,2023-07-21,esms,tpb_01,VISA CREDIT GOLD EVO CARD (9024),GOLD,mobifone,2023-08-18,2023-08-18,900000.0,<NA>
1,tpbank,3519374,14223442,SMS WITH OFFER,AVZZvxUuxmayAxuTwzSMCMSRHgga92+JfG0HUvQ=,<NA>,aRFe_O5a,2023-07-26,esms,tpb_01,VISA CREDIT GOLD EVO CARD (9024),GOLD,mobifone,2023-08-01,2023-08-04,900000.0,<NA>
2,tpbank,3520607,14224060,SMS WITH OFFER,Aes2QNkwisrMSVy0+5qrQ8LZHlb18foqaDWax7s=,<NA>,YRFfAwNu,2023-08-01,esms,tpb_01,VISA CREDIT GOLD EVO CARD (9024),GOLD,mobifone,2023-08-02,2023-08-04,900000.0,<NA>
3,tpbank,3519602,14223540,SMS WITH OFFER,AXUr79XzNUQfoDl4RP2HJrCwxfW4Gn+INzfYw3o=,<NA>,cRFe_YM_,2023-07-27,esms,tpb_01,VISA CREDIT GOLD EVO CARD (9024),GOLD,mobifone,2023-08-09,2023-08-16,900000.0,<NA>
4,tpbank,3516677,14222342,SMS WITH OFFER,AT1jNHQtfhCK4RwwlejWzJ5aMgUFv+N5DrutT60=,<NA>,QiK97rHg,2023-07-20,esms,tpb_01,VISA CREDIT GOLD EVO CARD (9024),GOLD,mobifone,2023-08-05,NaT,900000.0,<NA>


In [44]:
disbursal_df.count()

bank_code            25
telco_lead_id        25
lead_id              25
lead_source          25
phone_number         25
nid                   0
lead_unique_token    25
lead_created_date    25
channel              25
product_code         25
card_name            25
card_class           25
telco_code           25
issued_date          25
activated_date       22
commission           25
other                 0
dtype: int64

In [45]:
file_path = f"{path}{recon_month}/tpb_leadgen_recon_{recon_month}.csv"
disbursal_df.to_csv(file_path, index=False)

In [46]:
file_path: Path = Path(file_path)

# Send success message to channel
clientSlack.chat_postMessage(**{"text": f"<@{bot_id}> update leadgen card TPB {recon_month}"}, channel=channel_id)

# Upload file to channel
filetype = file_path.suffix[1:]
filename = file_path.name
with open(file_path, "rb") as file:
	clientSlack.files_upload(channels=channel_id, file=file, title=filename, filetype=filetype)
